In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']='4'
import numpy as np
import torch
import open3d as o3d
from tqdm import tqdm

from Closest_Point_on_Surface import DirDist_P2P
from utils.LieAlgebra import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2024-04-08 16:21:26,976 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/data/siyuren_21/anaconda3/envs/pugeo/lib/python3.8/site-packages/pytorch3d/renderer/opengl/__init__.py:27: UserWarning: Can't import pucuda.gl, not importing MeshRasterizerOpenGL.
  warnings.warn("Can't import pucuda.gl, not importing MeshRasterizerOpenGL.")


Define the loss functoin. Here we set up_ratio=10, K=5, std=0.05, and beta=10.

In [2]:
loss_func=DirDist_P2P(10,5,0.05,True,10)

Optimize the transformation between source and target points by minimizing the distance between them.

In [3]:
n_iters=2000

src_pcd=o3d.io.read_point_cloud('demo_data/rigid_registration/src.ply')
tgt_pcd=o3d.io.read_point_cloud('demo_data/rigid_registration/tgt.ply')

src=np.array(src_pcd.points)
tgt=np.array(tgt_pcd.points)

src=torch.from_numpy(src).cuda().float()
tgt=torch.from_numpy(tgt).cuda().float()

Reconstruction=Reconstruction_point(zero_init=True).cuda()
optimizer=torch.optim.Adam(Reconstruction.parameters(),lr=2e-2)
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,n_iters,2e-4)

for _ in tqdm(range(n_iters)):
    transformed_src,_=Reconstruction(src,None)
    loss=loss_func(transformed_src.unsqueeze(0),tgt.unsqueeze(0))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

trans = Reconstruction.Transform()
transforms = np.eye(4, 4)
transforms[:3, :3] = trans[0].detach().cpu().numpy()
transforms[:3, 3] = trans[1].detach().cpu().numpy()

transforms[:3,:3]=transforms[:3,:3].T

del optimizer, Reconstruction

print('the transformation is: ')
print(transforms)

100%|██████████| 2000/2000 [00:28<00:00, 70.10it/s]

the transformation is: 
[[ 0.97160226 -0.05002695  0.23121469  0.15126088]
 [ 0.05733332  0.99804282 -0.02496526  0.2017542 ]
 [-0.22951268  0.0375147   0.97256845 -0.27955729]
 [ 0.          0.          0.          1.        ]]
